In [2]:
import os
from dotenv import load_dotenv
from supabase import create_client
import pandas as pd
from datetime import date
import joblib

# Note: sklearn imports have been moved to a protective import cell to avoid binary-incompatibility crashes



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/home/sevan/ana

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/home/sevan/ana

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/home/sevan/ana

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/home/sevan/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/home/sevan/ana

AttributeError: _ARRAY_API not found

In [3]:
# Protective sklearn import: detect binary incompatibility and provide safe fallbacks
import numpy as np
SKLEARN_OK = True
try:
    # try to import critical sklearn pieces used by the notebook
    from sklearn.metrics.pairwise import cosine_similarity
    from sklearn.preprocessing import MinMaxScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report
except Exception as _e:
    print('Warning: sklearn import failed (binary mismatch). Falling back to safe substitutes for notebook execution:', _e)
    SKLEARN_OK = False

    # fallback cosine_similarity (single-pair) used by our code
    def cosine_similarity(a, b):
        a = np.array(a).reshape(1, -1)
        b = np.array(b).reshape(1, -1)
        num = (a * b).sum(axis=1)
        den = (np.linalg.norm(a, axis=1) * np.linalg.norm(b, axis=1)) + 1e-9
        return (num / den).reshape(1, 1)

    # lightweight MinMaxScaler substitute (only fit_transform needed)
    class MinMaxScaler:
        def fit_transform(self, X):
            import numpy as _np
            X = _np.array(X, dtype=float)
            if X.size == 0:
                return X
            mins = _np.nanmin(X, axis=0)
            maxs = _np.nanmax(X, axis=0)
            rng = maxs - mins
            rng[rng == 0] = 1.0
            out = (_np.nan_to_num(X) - mins) / rng
            return out.tolist()

    # lightweight LabelEncoder substitute
    class LabelEncoder:
        def fit_transform(self, y):
            import numpy as _np
            self.classes_, y_int = _np.unique(y, return_inverse=True)
            return y_int

    # RandomForestClassifier fallback that raises clear error if used (prefer skipping supervised path)
    class RandomForestClassifier:
        def __init__(self, *args, **kwargs):
            raise RuntimeError('RandomForestClassifier not available because sklearn import failed')

    # train_test_split stub
    def train_test_split(X, y, test_size=0.2, random_state=None, stratify=None):
        # trivial split: return full X as both train and test to avoid crashes
        return X, X, y, y

    # classification_report stub
    def classification_report(y_true, y_pred):
        return 'classification_report unavailable (sklearn missing)'

print('SKLEARN_OK =', SKLEARN_OK)

SKLEARN_OK = False


In [4]:
# --- Charger les variables d’environnement ---
load_dotenv()
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

# --- Connexion Supabase ---
try:
    supabase = create_client(url, key)
    print("Connexion à Supabase établie avec succès")
except Exception as e:
    print(f"Erreur lors de la connexion ou de la requête à Supabase : {str(e)}")

Connexion à Supabase établie avec succès


In [5]:
# --- Liste des tables utiles à interroger ---
tables = [
    "users",
    "applications",
    "job_offers",
    "seeg_agents",
    "candidate_profiles",
    "application_documents"
]

# --- Dictionnaire pour stocker les DataFrames ---
dataframes = {}

for table_name in tables:
    try:
        response = supabase.table(table_name).select("*").limit(100).execute()
        if response.data:
            df = pd.DataFrame(response.data)
            dataframes[table_name] = df
            print(f"✅ Table '{table_name}' récupérée avec {len(df)} lignes")
        else:
            print(f"⚠️ Table '{table_name}' vide ou RLS bloque l'accès")
    except Exception as e:
        print(f"❌ Erreur sur la table '{table_name}': {e}")

✅ Table 'users' récupérée avec 100 lignes
✅ Table 'applications' récupérée avec 52 lignes
✅ Table 'job_offers' récupérée avec 17 lignes
✅ Table 'seeg_agents' récupérée avec 100 lignes
✅ Table 'candidate_profiles' récupérée avec 86 lignes
✅ Table 'application_documents' récupérée avec 100 lignes


In [6]:
# Accéder au DataFrame de la table 'users'
df_users = dataframes.get('users')
df_app = dataframes.get('applications')
df_jobs = dataframes.get('job_offers')
df_agents = dataframes.get('seeg_agents')
df_profiles = dataframes.get('candidate_profiles')
df_docs = dataframes.get('documents')
df_app_docs = dataframes.get('application_documents')

# Application 

In [7]:
df_app.head()

,id,candidate_id,job_offer_id,cover_letter,status,motivation,availability_start,reference_contacts,created_at,updated_at,...,mtp_metier_q1,mtp_metier_q2,mtp_metier_q3,mtp_talent_q1,mtp_talent_q2,mtp_talent_q3,mtp_paradigme_q1,mtp_paradigme_q2,mtp_paradigme_q3,mtp_answers
0,c60e6947-4b66-460b-b4ea-69229e193b90,4fa0fad9-d98f-4849-9ad4-68def0ed2074,f426f59a-97b5-4bd1-b260-165dec5362bf,None,candidature,None,None,Laurent BARRERE\nDirecteur Général de SOGAFRIC...,2025-08-25T14:47:21.710145+00:00,2025-08-25T14:47:21.710145+00:00,...,None,None,None,None,None,None,None,None,None,{'metier': ['- Ingénierie énergétique - Mainte...
1,be95722d-8219-4b0e-acc9-50d032952b79,9f259121-b516-417f-8087-51a2fdf6d962,41112afb-d778-4c2b-bb86-f9ee3b6ca5a4,None,candidature,None,None,"Damien Madeleine, CAPGEMINI et Gérant de socié...",2025-08-25T15:26:23.482148+00:00,2025-08-25T15:26:23.482148+00:00,...,None,None,None,None,None,None,None,None,None,"{'metier': ['Secteur : Énergie (B2B, B2C, syst..."
2,6f0a3290-0204-4662-8902-3991c8fcb0ab,4dc6bf49-7813-49fd-a764-b8a8fbe33995,c34e8321-c9ab-4884-902d-ce19d977bd90,None,candidature,None,None,,2025-08-25T15:43:26.80122+00:00,2025-08-25T15:43:26.80122+00:00,...,None,None,None,None,None,None,None,None,None,{'metier': ['les problématiques se trouvent au...
3,ddebdf20-22ce-4da9-a490-c94516fb41fe,2f11d213-49f8-4407-ae5b-22cc99d3706b,737260aa-b155-443f-b21d-58bcbf8d151a,None,candidature,None,None,,2025-08-25T17:23:14.63456+00:00,2025-08-25T17:23:14.63456+00:00,...,None,None,None,None,None,None,None,None,None,{'metier': ['La méconnaissance des installatio...
4,d8321fde-fe23-4f10-863e-3102e625c14b,1e503472-4003-4941-ad30-8b1418f80170,707a8b92-4043-44c9-8a52-6c5b09cbb74c,None,candidature,None,None,,2025-08-25T18:23:11.227281+00:00,2025-08-25T18:23:11.227281+00:00,...,None,None,None,None,None,None,None,None,None,{'metier': ['- sur le plan technique: Les prob...


In [8]:
df_app.shape

(52, 22)

In [9]:
df_app.columns

Index(['id', 'candidate_id', 'job_offer_id', 'cover_letter', 'status',
       'motivation', 'availability_start', 'reference_contacts', 'created_at',
       'updated_at', 'url_lettre_integrite', 'url_idee_projet',
       'mtp_metier_q1', 'mtp_metier_q2', 'mtp_metier_q3', 'mtp_talent_q1',
       'mtp_talent_q2', 'mtp_talent_q3', 'mtp_paradigme_q1',
       'mtp_paradigme_q2', 'mtp_paradigme_q3', 'mtp_answers'],
      dtype='object')

In [10]:
# Suppression des colonnes non nécessaires
delete_columns = ['cover_letter', 'motivation', 'availability_start', 'url_lettre_integrite', 'url_idee_projet', 'mtp_metier_q1', 'mtp_metier_q2', 'mtp_metier_q3', 'mtp_talent_q1', 'mtp_talent_q2', 'mtp_talent_q3', 'mtp_paradigme_q1', 'mtp_paradigme_q2', 'mtp_paradigme_q3', 'created_at','updated_at']
df_app = df_app.drop(columns=delete_columns)

In [11]:
df_app.columns

Index(['id', 'candidate_id', 'job_offer_id', 'status', 'reference_contacts',
       'mtp_answers'],
      dtype='object')

In [12]:
df_app.head(10)

,id,candidate_id,job_offer_id,status,reference_contacts,mtp_answers
0,c60e6947-4b66-460b-b4ea-69229e193b90,4fa0fad9-d98f-4849-9ad4-68def0ed2074,f426f59a-97b5-4bd1-b260-165dec5362bf,candidature,Laurent BARRERE\nDirecteur Général de SOGAFRIC...,{'metier': ['- Ingénierie énergétique - Mainte...
1,be95722d-8219-4b0e-acc9-50d032952b79,9f259121-b516-417f-8087-51a2fdf6d962,41112afb-d778-4c2b-bb86-f9ee3b6ca5a4,candidature,"Damien Madeleine, CAPGEMINI et Gérant de socié...","{'metier': ['Secteur : Énergie (B2B, B2C, syst..."
2,6f0a3290-0204-4662-8902-3991c8fcb0ab,4dc6bf49-7813-49fd-a764-b8a8fbe33995,c34e8321-c9ab-4884-902d-ce19d977bd90,candidature,,{'metier': ['les problématiques se trouvent au...
3,ddebdf20-22ce-4da9-a490-c94516fb41fe,2f11d213-49f8-4407-ae5b-22cc99d3706b,737260aa-b155-443f-b21d-58bcbf8d151a,candidature,,{'metier': ['La méconnaissance des installatio...
4,d8321fde-fe23-4f10-863e-3102e625c14b,1e503472-4003-4941-ad30-8b1418f80170,707a8b92-4043-44c9-8a52-6c5b09cbb74c,candidature,,{'metier': ['- sur le plan technique: Les prob...
5,e89bab46-05ff-4898-842d-2d002d8b2d80,eb8fe2c4-0093-4cf3-9731-4ddec79239e6,3dc73269-03b2-42cb-a608-2943ed45731b,candidature,"Pr Jean Jacques EKOMIE, Recteur de l'UOB, Tel....",{'metier': ['1/Mauvaise gestion du capital hum...
6,856a2c7d-4418-44c5-ab49-c29bfc8842d6,ec070d9c-812d-4816-85ff-8c2efc34919b,436331ac-ce6e-428c-bbb6-dad1841f8e85,candidature,,{'metier': ['- Connaissances et maitrise des r...
7,3fdf0cf6-5efb-426f-80ba-b86c9e0a7833,086d60e7-0bdc-446d-80ef-d1cda6bb5126,391ccc64-2ee5-464f-942d-26df1719eeb6,candidature,,{'metier': ['Insuffisance de la la production ...
8,dd1dd88c-e970-46b0-93c7-9cae866458c6,d1cf65ac-366d-4b71-ae69-3ff8956eb421,f426f59a-97b5-4bd1-b260-165dec5362bf,candidature,,{'metier': ['1. Problématiques clés identifiée...
9,834cca94-a92e-4cf0-9f14-bbf16814313b,ef01118b-2f71-41f0-a1a7-cf7418505f0f,41112afb-d778-4c2b-bb86-f9ee3b6ca5a4,candidature,,{'metier': ['les problématiques que j'identifi...


In [13]:
df_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  52 non-null     object
 1   candidate_id        52 non-null     object
 2   job_offer_id        52 non-null     object
 3   status              52 non-null     object
 4   reference_contacts  52 non-null     object
 5   mtp_answers         52 non-null     object
dtypes: object(6)
memory usage: 2.6+ KB


# Users

In [14]:
df_users.head()

,id,email,matricule,role,first_name,last_name,phone,date_of_birth,created_at,updated_at,sexe
0,ff8bda17-9bbe-46b6-8e45-33218393f862,okalamarchic@gmail.com,None,candidat,okalamar,chic,None,2007-12-13,2025-08-24T23:15:15.793555+00:00,2025-08-24T23:15:15.793555+00:00,None
1,dadddad4-2b0f-47ab-b058-2e6150dc287f,aokouaghe@seeg-gabon.com,6178,candidat,Armel,OKOUAGHE,+24177573974,None,2025-08-25T07:35:08.431701+00:00,2025-08-25T07:35:08.431701+00:00,None
2,a4171a4a-a215-402f-9617-37d46a180cfa,arnymar01@gmail.com,6064,candidat,Armand Rodrigue,MIHINDOU PAGA,+24177854596,None,2025-08-25T09:40:38.344953+00:00,2025-08-25T09:40:38.344953+00:00,None
3,68043e8d-d20f-4fb0-9649-0f7a22900713,melky@cnx4-0.com,None,recruiter,melky,,None,None,2025-08-20T17:42:42.090781+00:00,2025-08-20T17:42:42.090781+00:00,None
4,4dc6bf49-7813-49fd-a764-b8a8fbe33995,sogoula@seeg-gabon.com,6010,candidat,steeve,ogoula ndabendjet,077295132,None,2025-08-25T10:13:53.782564+00:00,2025-08-25T10:13:53.782564+00:00,None


In [15]:
df_users.shape

(100, 11)

In [16]:
df_users.columns

Index(['id', 'email', 'matricule', 'role', 'first_name', 'last_name', 'phone',
       'date_of_birth', 'created_at', 'updated_at', 'sexe'],
      dtype='object')

In [17]:
delete_columns = ['matricule', 'phone', 'date_of_birth', 'created_at', 'updated_at', 'sexe']
df_users = df_users.drop(columns=delete_columns)

In [18]:
df_users.columns

Index(['id', 'email', 'role', 'first_name', 'last_name'], dtype='object')

In [19]:
df_users.head()

,id,email,role,first_name,last_name
0,ff8bda17-9bbe-46b6-8e45-33218393f862,okalamarchic@gmail.com,candidat,okalamar,chic
1,dadddad4-2b0f-47ab-b058-2e6150dc287f,aokouaghe@seeg-gabon.com,candidat,Armel,OKOUAGHE
2,a4171a4a-a215-402f-9617-37d46a180cfa,arnymar01@gmail.com,candidat,Armand Rodrigue,MIHINDOU PAGA
3,68043e8d-d20f-4fb0-9649-0f7a22900713,melky@cnx4-0.com,recruiter,melky,
4,4dc6bf49-7813-49fd-a764-b8a8fbe33995,sogoula@seeg-gabon.com,candidat,steeve,ogoula ndabendjet


In [20]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          100 non-null    object
 1   email       100 non-null    object
 2   role        100 non-null    object
 3   first_name  100 non-null    object
 4   last_name   100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


# Profiles de candidature

In [21]:
df_profiles.head()

,id,user_id,current_position,current_department,years_experience,education,availability,expected_salary_min,expected_salary_max,skills,cv_url,created_at,updated_at,gender,birth_date,linkedin_url,portfolio_url,address
0,6643d6f8-b423-49c0-a0dd-e65da33d1427,60fcb996-b80f-46a5-b580-02d356c2a960,"Ex ""Responsable de Division"" Communication Ext...",None,14.0,None,None,None,None,None,None,2025-08-27T13:52:51.917507+00:00,2025-08-27T13:52:51.917507+00:00,femme,1970-11-27,None,None,AKANDA
1,52b2038b-31cb-4410-b08d-4c965c028d15,086d60e7-0bdc-446d-80ef-d1cda6bb5126,Chef de Département Administration & Qualité,None,0.0,None,None,None,None,None,None,2025-08-26T15:45:52.39066+00:00,2025-08-26T15:45:52.39066+00:00,homme,1976-11-06,None,None,Akanda les kikinous\nCommune de Libreville
2,faa404a6-be00-4e52-b846-a3334eae33ab,2f11d213-49f8-4407-ae5b-22cc99d3706b,Chef du Pôle Expertise Maintenance Spécialisée...,None,25.0,None,None,None,None,None,None,2025-08-26T15:45:52.39066+00:00,2025-08-26T15:45:52.39066+00:00,homme,1974-03-14,None,None,None
3,5ffefe0b-4ec3-49e5-9c2e-9dafbd9a863c,680aafd0-21bc-4c70-9e06-af33810aba96,agent technique analyse eau,None,11.0,None,None,None,None,None,None,2025-08-27T08:45:17.100014+00:00,2025-08-27T08:45:17.100014+00:00,homme,1981-08-23,None,None,LIBREVILLE
4,920a9e21-67da-48e7-8ddf-54cab5593a3e,9f259121-b516-417f-8087-51a2fdf6d962,Directeur des Systèmes d’Information,None,15.0,None,None,None,None,None,None,2025-08-26T15:45:52.39066+00:00,2025-08-26T15:45:52.39066+00:00,homme,1984-07-17,None,None,"Mveng-Ayong, Akanda"


In [22]:
df_profiles.columns

Index(['id', 'user_id', 'current_position', 'current_department',
       'years_experience', 'education', 'availability', 'expected_salary_min',
       'expected_salary_max', 'skills', 'cv_url', 'created_at', 'updated_at',
       'gender', 'birth_date', 'linkedin_url', 'portfolio_url', 'address'],
      dtype='object')

In [23]:
delete_columns = ['current_department', 'education', 'availability', 'expected_salary_min', 'skills', 'cv_url', 'created_at', 'updated_at', 'linkedin_url', 'portfolio_url', 'expected_salary_max']
df_profiles = df_profiles.drop(columns=delete_columns)

In [24]:
df_profiles.columns

Index(['id', 'user_id', 'current_position', 'years_experience', 'gender',
       'birth_date', 'address'],
      dtype='object')

In [25]:
df_profiles.head()

,id,user_id,current_position,years_experience,gender,birth_date,address
0,6643d6f8-b423-49c0-a0dd-e65da33d1427,60fcb996-b80f-46a5-b580-02d356c2a960,"Ex ""Responsable de Division"" Communication Ext...",14.0,femme,1970-11-27,AKANDA
1,52b2038b-31cb-4410-b08d-4c965c028d15,086d60e7-0bdc-446d-80ef-d1cda6bb5126,Chef de Département Administration & Qualité,0.0,homme,1976-11-06,Akanda les kikinous\nCommune de Libreville
2,faa404a6-be00-4e52-b846-a3334eae33ab,2f11d213-49f8-4407-ae5b-22cc99d3706b,Chef du Pôle Expertise Maintenance Spécialisée...,25.0,homme,1974-03-14,None
3,5ffefe0b-4ec3-49e5-9c2e-9dafbd9a863c,680aafd0-21bc-4c70-9e06-af33810aba96,agent technique analyse eau,11.0,homme,1981-08-23,LIBREVILLE
4,920a9e21-67da-48e7-8ddf-54cab5593a3e,9f259121-b516-417f-8087-51a2fdf6d962,Directeur des Systèmes d’Information,15.0,homme,1984-07-17,"Mveng-Ayong, Akanda"


In [26]:
df_profiles['years_experience'] = df_profiles['years_experience'].astype('Int64')
df_profiles["birth_date"] = pd.to_datetime(df_profiles["birth_date"]).dt.date

In [27]:
df_profiles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                86 non-null     object
 1   user_id           86 non-null     object
 2   current_position  85 non-null     object
 3   years_experience  85 non-null     Int64 
 4   gender            83 non-null     object
 5   birth_date        85 non-null     object
 6   address           75 non-null     object
dtypes: Int64(1), object(6)
memory usage: 4.9+ KB


In [28]:
df_profiles.head()

,id,user_id,current_position,years_experience,gender,birth_date,address
0,6643d6f8-b423-49c0-a0dd-e65da33d1427,60fcb996-b80f-46a5-b580-02d356c2a960,"Ex ""Responsable de Division"" Communication Ext...",14,femme,1970-11-27,AKANDA
1,52b2038b-31cb-4410-b08d-4c965c028d15,086d60e7-0bdc-446d-80ef-d1cda6bb5126,Chef de Département Administration & Qualité,0,homme,1976-11-06,Akanda les kikinous\nCommune de Libreville
2,faa404a6-be00-4e52-b846-a3334eae33ab,2f11d213-49f8-4407-ae5b-22cc99d3706b,Chef du Pôle Expertise Maintenance Spécialisée...,25,homme,1974-03-14,None
3,5ffefe0b-4ec3-49e5-9c2e-9dafbd9a863c,680aafd0-21bc-4c70-9e06-af33810aba96,agent technique analyse eau,11,homme,1981-08-23,LIBREVILLE
4,920a9e21-67da-48e7-8ddf-54cab5593a3e,9f259121-b516-417f-8087-51a2fdf6d962,Directeur des Systèmes d’Information,15,homme,1984-07-17,"Mveng-Ayong, Akanda"


In [29]:
df_profiles.shape

(86, 7)

In [30]:
# Enrichissement du DataFrame des candidatures avec les informations des profils de candidats

today = date.today()

df_profiles["age"] = df_profiles["birth_date"].apply(
    lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)) if pd.notnull(x) else None
)
df_profiles['age'] = df_profiles['age'].astype('Int64')

In [31]:
df_profiles.head()

,id,user_id,current_position,years_experience,gender,birth_date,address,age
0,6643d6f8-b423-49c0-a0dd-e65da33d1427,60fcb996-b80f-46a5-b580-02d356c2a960,"Ex ""Responsable de Division"" Communication Ext...",14,femme,1970-11-27,AKANDA,54
1,52b2038b-31cb-4410-b08d-4c965c028d15,086d60e7-0bdc-446d-80ef-d1cda6bb5126,Chef de Département Administration & Qualité,0,homme,1976-11-06,Akanda les kikinous\nCommune de Libreville,48
2,faa404a6-be00-4e52-b846-a3334eae33ab,2f11d213-49f8-4407-ae5b-22cc99d3706b,Chef du Pôle Expertise Maintenance Spécialisée...,25,homme,1974-03-14,None,51
3,5ffefe0b-4ec3-49e5-9c2e-9dafbd9a863c,680aafd0-21bc-4c70-9e06-af33810aba96,agent technique analyse eau,11,homme,1981-08-23,LIBREVILLE,44
4,920a9e21-67da-48e7-8ddf-54cab5593a3e,9f259121-b516-417f-8087-51a2fdf6d962,Directeur des Systèmes d’Information,15,homme,1984-07-17,"Mveng-Ayong, Akanda",41


# Description de l'offre

#

In [32]:
df_jobs.head()

,id,recruiter_id,title,description,location,contract_type,department,salary_min,salary_max,requirements,...,created_at,updated_at,profile,categorie_metier,date_limite,reporting_line,job_grade,salary_note,start_date,responsibilities
0,707a8b92-4043-44c9-8a52-6c5b09cbb74c,248110aa-780e-4043-8e12-9dfe0047613c,Coordonnateur des Régions,<p> <strong>&nbsp;</strong></p><p>Le coordonn...,Libreville,CDI avec période d'essai,None,None,None,"[FORMATION :, BAC + 5 en Génie électrique, Gén...",...,2025-08-22T11:16:15.315154+00:00,2025-08-22T11:16:15.315154+00:00,<p><strong>FORMATION : </strong></p><p>BAC + 5...,Cadre directeur,2025-08-31T00:00:00+00:00,Administrateur Directeur Général,None,Grille salariale et avantages de la SEEG,2025-09-15,[Le coordonnateur des régions est l’animateur ...
1,737260aa-b155-443f-b21d-58bcbf8d151a,248110aa-780e-4043-8e12-9dfe0047613c,Directeur Technique Eau,<p><strong>Pilotage technique global </strong>...,Libreville,CDI avec période d'essai,None,None,None,"[FORMATION :, BAC + 5 Ingénieur en génie de l’...",...,2025-08-22T11:16:51.722979+00:00,2025-08-22T11:16:51.722979+00:00,<p><strong>FORMATION : </strong></p><p>BAC + 5...,Cadre directeur,2025-08-31T00:00:00+00:00,Chef Département Eau,None,Grille salariale et avantages de la SEEG,2025-09-15,"[Pilotage technique global, Assurer la continu..."
2,391ccc64-2ee5-464f-942d-26df1719eeb6,248110aa-780e-4043-8e12-9dfe0047613c,Chef de Département Eau,<p>-&nbsp;Assurer la bonne répartition des res...,Libreville,CDI avec période d'essai,None,None,None,"[FORMATION, BAC + 5 Ingénieur en génie de l’ea...",...,2025-08-22T10:36:58.179828+00:00,2025-08-22T10:36:58.179828+00:00,<p><strong>FORMATION </strong></p><p>BAC + 5 I...,Cadre directeur,2025-08-31T00:00:00+00:00,Administrateur Directeur Général,None,Grille salariale et avantages de la SEEG,2025-09-15,[Assurer la bonne répartition des ressources h...
3,f4b7df17-95ac-4252-b057-e3503d072364,248110aa-780e-4043-8e12-9dfe0047613c,Chef de Département Electricité,<p>-&nbsp;Assurer la bonne répartition des res...,Libreville,CDI avec période d'essai,None,None,None,"[FORMATION :, BAC + 5 en électrotechnique, gén...",...,2025-08-22T10:38:32.29489+00:00,2025-08-22T10:38:32.29489+00:00,<p><strong>FORMATION : </strong></p><p>BAC + 5...,Cadre directeur,2025-08-31T00:00:00+00:00,Administrateur Directeur Général,None,Grille salariale et avantages de la SEEG,2025-09-15,[Assurer la bonne répartition des ressources h...
4,ee373d45-efcb-43e0-89ef-0983ac50d8da,248110aa-780e-4043-8e12-9dfe0047613c,Directeur Exploitation Eau,<p><strong><u>Volet Pilotage des opérations d’...,Libreville,CDI avec période d'essai,None,None,None,[],...,2025-08-22T11:17:24.782973+00:00,2025-08-22T11:17:24.782973+00:00,<p><strong>Formation&nbsp;: </strong>BAC + 5 /...,Cadre directeur,2025-08-31T00:00:00+00:00,Chef de Département Eau,None,Grille salariale et avantages de la SEEG,2025-09-15,[]


In [33]:
df_jobs.columns

Index(['id', 'recruiter_id', 'title', 'description', 'location',
       'contract_type', 'department', 'salary_min', 'salary_max',
       'requirements', 'benefits', 'status', 'application_deadline',
       'created_at', 'updated_at', 'profile', 'categorie_metier',
       'date_limite', 'reporting_line', 'job_grade', 'salary_note',
       'start_date', 'responsibilities'],
      dtype='object')

In [34]:
delete_columns = ['department', 'salary_min', 'salary_max', 'benefits', 'application_deadline', 'created_at', 'updated_at', 'start_date', 'job_grade']
df_jobs = df_jobs.drop(columns=delete_columns)

In [35]:
df_jobs.head()

,id,recruiter_id,title,description,location,contract_type,requirements,status,profile,categorie_metier,date_limite,reporting_line,salary_note,responsibilities
0,707a8b92-4043-44c9-8a52-6c5b09cbb74c,248110aa-780e-4043-8e12-9dfe0047613c,Coordonnateur des Régions,<p> <strong>&nbsp;</strong></p><p>Le coordonn...,Libreville,CDI avec période d'essai,"[FORMATION :, BAC + 5 en Génie électrique, Gén...",active,<p><strong>FORMATION : </strong></p><p>BAC + 5...,Cadre directeur,2025-08-31T00:00:00+00:00,Administrateur Directeur Général,Grille salariale et avantages de la SEEG,[Le coordonnateur des régions est l’animateur ...
1,737260aa-b155-443f-b21d-58bcbf8d151a,248110aa-780e-4043-8e12-9dfe0047613c,Directeur Technique Eau,<p><strong>Pilotage technique global </strong>...,Libreville,CDI avec période d'essai,"[FORMATION :, BAC + 5 Ingénieur en génie de l’...",active,<p><strong>FORMATION : </strong></p><p>BAC + 5...,Cadre directeur,2025-08-31T00:00:00+00:00,Chef Département Eau,Grille salariale et avantages de la SEEG,"[Pilotage technique global, Assurer la continu..."
2,391ccc64-2ee5-464f-942d-26df1719eeb6,248110aa-780e-4043-8e12-9dfe0047613c,Chef de Département Eau,<p>-&nbsp;Assurer la bonne répartition des res...,Libreville,CDI avec période d'essai,"[FORMATION, BAC + 5 Ingénieur en génie de l’ea...",active,<p><strong>FORMATION </strong></p><p>BAC + 5 I...,Cadre directeur,2025-08-31T00:00:00+00:00,Administrateur Directeur Général,Grille salariale et avantages de la SEEG,[Assurer la bonne répartition des ressources h...
3,f4b7df17-95ac-4252-b057-e3503d072364,248110aa-780e-4043-8e12-9dfe0047613c,Chef de Département Electricité,<p>-&nbsp;Assurer la bonne répartition des res...,Libreville,CDI avec période d'essai,"[FORMATION :, BAC + 5 en électrotechnique, gén...",active,<p><strong>FORMATION : </strong></p><p>BAC + 5...,Cadre directeur,2025-08-31T00:00:00+00:00,Administrateur Directeur Général,Grille salariale et avantages de la SEEG,[Assurer la bonne répartition des ressources h...
4,ee373d45-efcb-43e0-89ef-0983ac50d8da,248110aa-780e-4043-8e12-9dfe0047613c,Directeur Exploitation Eau,<p><strong><u>Volet Pilotage des opérations d’...,Libreville,CDI avec période d'essai,[],active,<p><strong>Formation&nbsp;: </strong>BAC + 5 /...,Cadre directeur,2025-08-31T00:00:00+00:00,Chef de Département Eau,Grille salariale et avantages de la SEEG,[]


In [36]:
df_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                17 non-null     object
 1   recruiter_id      17 non-null     object
 2   title             17 non-null     object
 3   description       17 non-null     object
 4   location          17 non-null     object
 5   contract_type     17 non-null     object
 6   requirements      16 non-null     object
 7   status            17 non-null     object
 8   profile           17 non-null     object
 9   categorie_metier  17 non-null     object
 10  date_limite       17 non-null     object
 11  reporting_line    17 non-null     object
 12  salary_note       16 non-null     object
 13  responsibilities  16 non-null     object
dtypes: object(14)
memory usage: 2.0+ KB


In [37]:
df_jobs.columns

Index(['id', 'recruiter_id', 'title', 'description', 'location',
       'contract_type', 'requirements', 'status', 'profile',
       'categorie_metier', 'date_limite', 'reporting_line', 'salary_note',
       'responsibilities'],
      dtype='object')

# Agents

In [38]:
df_agents.head()

,matricule,nom,prenom,created_at
0,4517,ETOUGHE EFAGHE,Joseph,2025-08-21T16:35:26.788015+00:00
1,4547,MISSONI,Edouard,2025-08-21T16:35:26.788015+00:00
2,4581,MBONGO,Edouard,2025-08-21T16:35:26.788015+00:00
3,4640,MOUCKAKI BOUMAH,Augustin,2025-08-21T16:35:26.788015+00:00
4,4642,ISSAMOROUNGOU,Alfred Gustave,2025-08-21T16:35:26.788015+00:00


In [39]:
df_agents.columns

Index(['matricule', 'nom', 'prenom', 'created_at'], dtype='object')

In [40]:
df_agents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   matricule   100 non-null    int64 
 1   nom         100 non-null    object
 2   prenom      100 non-null    object
 3   created_at  100 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [41]:
string_columns = ['nom', 'prenom']
df_agents[string_columns] = df_agents[string_columns].astype(str)
df_agents['matricule'] = df_agents['matricule'].astype(int)

In [42]:
df_agents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   matricule   100 non-null    int64 
 1   nom         100 non-null    object
 2   prenom      100 non-null    object
 3   created_at  100 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [43]:
df_agents.columns

Index(['matricule', 'nom', 'prenom', 'created_at'], dtype='object')

# Application documentaire

In [44]:
df_app_docs.head()

,id,application_id,document_type,file_name,file_url,file_size,uploaded_at
0,204db6aa-4f1e-4d01-8d71-c6ce856ba79c,d2d29fe8-80fd-4100-bb9d-b55990d6e36f,certificate,CIMEF.pdf,692a8572-a588-4df2-a2b0-7b81d7c86e1b/additiona...,1164858,2025-08-27T13:00:48.802466+00:00
1,1a808894-98e2-4a0e-8b43-5c81eb3e525c,d2d29fe8-80fd-4100-bb9d-b55990d6e36f,certificate,IDECA.pdf,692a8572-a588-4df2-a2b0-7b81d7c86e1b/additiona...,1010096,2025-08-27T13:00:48.802466+00:00
2,b5decfcc-e8af-432d-85d5-37069037af1c,d2d29fe8-80fd-4100-bb9d-b55990d6e36f,certificate,STBV.pdf,692a8572-a588-4df2-a2b0-7b81d7c86e1b/additiona...,1054476,2025-08-27T13:00:48.802466+00:00
3,63b78ff9-9317-4c23-93a0-59ee22ee7023,8c874b3d-a3bb-4600-b50f-172fae31bdbf,diploma,Diplome gestion des projets MMR.pdf,0ebf4291-111d-4fde-b5e6-a50d7c5f4bf6/certifica...,1427680,2025-08-27T13:01:47.232059+00:00
4,590be87b-aec7-447e-a78d-baf279f73b69,c60e6947-4b66-460b-b4ea-69229e193b90,cv,CV Davy Romaric MANGUENDZA.pdf,4fa0fad9-d98f-4849-9ad4-68def0ed2074/cv/175612...,386917,2025-08-25T14:47:23.013377+00:00


In [45]:
df_app_docs.columns

Index(['id', 'application_id', 'document_type', 'file_name', 'file_url',
       'file_size', 'uploaded_at'],
      dtype='object')

# Aggregation de tous les dataframes

# Modèle de Machine Learning pour l'Évaluation des Candidats

Ce modèle va nous permettre d'évaluer automatiquement les candidats en fonction de plusieurs critères :
1. Réponses MTP (Métier, Talent, Paradigme)
2. Adéquation avec la vision stratégique de la SEEG
3. Informations du CV et documents fournis
4. Expérience et compétences

In [ ]:
# Chargement du modèle français de spaCy
!python -m spacy download fr_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 6.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


: 

In [ ]:
# Import des bibliothèques nécessaires
import numpy as np
import tensorflow as tf
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy

# Téléchargement des ressources nécessaires
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Note: sklearn imports are now handled in the protective import cell above.


In [ ]:
# Chargement du modèle spaCy
nlp = spacy.load('fr_core_news_md')

def preprocess_text(text):
    """
    Prétraite le texte en effectuant les opérations suivantes :
    1. Tokenization
    2. Suppression des stop words
    3. Lemmatisation
    4. Extraction des entités nommées
    """
    if pd.isna(text):
        return ""
        
    # Traitement avec spaCy
    doc = nlp(str(text).lower())
    
    # Filtrage des tokens
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.is_alpha]
    
    # Extraction des entités nommées importantes
    entities = [ent.text for ent in doc.ents if ent.label_ in ['ORG', 'PRODUCT', 'GPE', 'SKILL']]
    
    return " ".join(tokens + entities)

In [ ]:
# Préparation des features pour le modèle

def prepare_candidate_features(row):
    """
    Prépare les features pour un candidat donné en combinant :
    - Réponses MTP
    - Informations du profil
    - Texte du CV et de la lettre de motivation
    """
    features = {
        # Questions MTP Métier
        'mtp_metier_score': np.mean([
            float(row.get('mtp_metier_q1', 0) or 0),
            float(row.get('mtp_metier_q2', 0) or 0),
            float(row.get('mtp_metier_q3', 0) or 0)
        ]),
        
        # Questions MTP Talent
        'mtp_talent_score': np.mean([
            float(row.get('mtp_talent_q1', 0) or 0),
            float(row.get('mtp_talent_q2', 0) or 0),
            float(row.get('mtp_talent_q3', 0) or 0)
        ]),
        
        # Questions MTP Paradigme
        'mtp_paradigme_score': np.mean([
            float(row.get('mtp_paradigme_q1', 0) or 0),
            float(row.get('mtp_paradigme_q2', 0) or 0),
            float(row.get('mtp_paradigme_q3', 0) or 0)
        ]),
        
        # Expérience professionnelle
        'years_experience': float(row.get('years_experience', 0) or 0),
        
        # Âge du candidat
        'age': float(row.get('age', 0) or 0),
    }
    
    return pd.Series(features)

# Application de la préparation des features sur le DataFrame des candidatures
df_features = df_app.merge(df_profiles, left_on='user_id', right_on='id', how='left')
candidate_features = df_features.apply(prepare_candidate_features, axis=1)

In [ ]:


# 1) Fonctions utilitaires

def safe_text_from_row(row, fields):
    """Récupère et concatène du texte depuis plusieurs colonnes si elles existent."""
    parts = []
    for f in fields:
        if f in row.index and pd.notna(row[f]):
            parts.append(str(row[f]))
    return "\n".join(parts)


def text_vector(text):
    """Retourne le vecteur spaCy normalisé pour un texte (ou vecteur nul si vide)."""
    if not text or str(text).strip() == "":
        return np.zeros((nlp.vocab.vectors_length,)) if hasattr(nlp.vocab, 'vectors_length') else np.zeros((300,))
    doc = nlp(text)
    vec = doc.vector
    # fallback shape
    if vec is None or len(vec) == 0:
        return np.zeros((nlp.vocab.vectors_length,)) if hasattr(nlp.vocab, 'vectors_length') else np.zeros((300,))
    return vec


def compute_job_vectors(df_jobs, text_fields=['description', 'job_description', 'title']):
    """Calcule un vecteur pour chaque offre d'emploi en essayant plusieurs champs."""
    def get_text_for_job(row):
        return safe_text_from_row(row, [f for f in text_fields if f in row.index])

    df_jobs = df_jobs.copy()
    df_jobs['job_text'] = df_jobs.apply(get_text_for_job, axis=1)
    df_jobs['job_vector'] = df_jobs['job_text'].apply(text_vector)
    return df_jobs


# 2) Préparer les textes candidats (CV, lettre, réponses MTP si texte)
text_candidate_fields = []
# champs probables - on essaie d'utiliser ceux présents
possible_candidate_text_fields = ['cover_letter', 'motivation', 'cv_text', 'cv_content', 'resume_text']
for f in possible_candidate_text_fields:
    if f in df_features.columns:
        text_candidate_fields.append(f)

# si df_app_docs contient du texte, on va l'agréger
if 'df_app_docs' in globals() and isinstance(df_app_docs, pd.DataFrame):
    doc_text_col = None
    # identifier une colonne texte dans df_app_docs
    for c in ['content', 'text', 'document_text', 'body']:
        if c in df_app_docs.columns:
            doc_text_col = c
            break
    if doc_text_col:
        # agréger par application_id
        if 'application_id' in df_app_docs.columns:
            docs_agg = df_app_docs.groupby('application_id')[doc_text_col].apply(lambda lst: '\n'.join(map(str, lst))).rename('docs_text')
            # joindre au df_features si possible
            if 'id' in df_features.columns:
                df_features = df_features.merge(docs_agg, left_on='id', right_index=True, how='left')
                text_candidate_fields.append('docs_text')

# Si aucun champ texte candidat trouvé, tenter des colonnes génériques dans df_profiles/df_app
for fallback in ['motivation', 'cover_letter', 'letter', 'summary']:
    if fallback in df_features.columns and fallback not in text_candidate_fields:
        text_candidate_fields.append(fallback)

# 3) Calculer vecteurs candidats
if len(text_candidate_fields) == 0:
    # si vraiment aucun texte, on créera des vecteurs nuls
    df_features['candidate_text'] = ''
else:
    df_features['candidate_text'] = df_features.apply(lambda r: safe_text_from_row(r, text_candidate_fields), axis=1)

# vecteurs
df_features['candidate_vector'] = df_features['candidate_text'].apply(text_vector)

# 4) Vecteur vision SEEG (modifier la variable ci-dessous avec le texte officiel si disponible)
seeg_vision_text = "Fournir de l'électricité et de l'eau de qualité, durablement, avec innovation, responsabilité sociale et excellence opérationnelle."
seeg_vision_vec = text_vector(seeg_vision_text)

# 5) Calculer vecteurs offres d'emploi
df_jobs = compute_job_vectors(df_jobs)

# 6) Associer chaque candidature à l'offre si possible (colonne job_offer_id / offer_id / job_id)
job_id_col_candidates = None
for c in ['job_offer_id', 'offer_id', 'job_id', 'job_offers_id']:
    if c in df_features.columns:
        job_id_col_candidates = c
        break

# join pour obtenir job_vector par candidature
if job_id_col_candidates:
    df_features = df_features.merge(df_jobs[['id', 'job_vector']], left_on=job_id_col_candidates, right_on='id', how='left', suffixes=('', '_job'))
    df_features['job_vector'] = df_features['job_vector'].apply(lambda v: v if isinstance(v, (np.ndarray, list)) else np.zeros_like(seeg_vision_vec))
else:
    # pas d'association, on met job_vector vide
    df_features['job_vector'] = [np.zeros_like(seeg_vision_vec)] * len(df_features)

# 7) Fonctions de similarité

def cosine_sim(a, b):
    try:
        if a is None or b is None:
            return 0.0
        a = np.array(a).reshape(1, -1)
        b = np.array(b).reshape(1, -1)
        return float(cosine_similarity(a, b)[0, 0])
    except Exception:
        return 0.0

# calculer similarités
df_features['sim_with_job'] = df_features.apply(lambda r: cosine_sim(r['candidate_vector'], r['job_vector']), axis=1)
df_features['sim_with_vision'] = df_features['candidate_vector'].apply(lambda v: cosine_sim(v, seeg_vision_vec))

df_features['candidate_text_len'] = df_features['candidate_text'].str.len().fillna(0)

# 8) Agrégation des features numériques existantes (MTP, experience, age)
# vérifier présence des colonnes MTP
mtp_cols = [c for c in ['mtp_metier_q1','mtp_metier_q2','mtp_metier_q3','mtp_talent_q1','mtp_talent_q2','mtp_talent_q3','mtp_paradigme_q1','mtp_paradigme_q2','mtp_paradigme_q3'] if c in df_features.columns]

if any('mtp_metier' in c for c in mtp_cols):
    # regenérer scores mtp si présent
    def safe_mean(cols, row):
        vals = []
        for cc in cols:
            if cc in row.index and pd.notna(row[cc]):
                try:
                    vals.append(float(row[cc]))
                except Exception:
                    pass
        return np.mean(vals) if vals else 0.0

    df_features['mtp_metier_score'] = df_features.apply(lambda r: safe_mean([c for c in mtp_cols if 'metier' in c], r), axis=1)
    df_features['mtp_talent_score'] = df_features.apply(lambda r: safe_mean([c for c in mtp_cols if 'talent' in c], r), axis=1)
    df_features['mtp_paradigme_score'] = df_features.apply(lambda r: safe_mean([c for c in mtp_cols if 'paradigme' in c], r), axis=1)
else:
    # utiliser colonnes existantes si le notebook a déjà créé ces
    for name in ['mtp_metier_score','mtp_talent_score','mtp_paradigme_score']:
        if name not in df_features.columns:
            df_features[name] = 0.0

# years_experience & age
for name in ['years_experience','age']:
    if name not in df_features.columns:
        df_features[name] = 0.0

# 9) Normalisation et score heuristique (si pas de label supervisé)
feature_cols_for_score = ['mtp_metier_score','mtp_talent_score','mtp_paradigme_score','years_experience','sim_with_job','sim_with_vision']
for c in feature_cols_for_score:
    if c not in df_features.columns:
        df_features[c] = 0.0

scaler = MinMaxScaler()
df_features['_score_norm'] = scaler.fit_transform(df_features[feature_cols_for_score].fillna(0))

# weighted sum across groups: moyenne des mtp + autres
# préparer vecteur normalisé (0..1) pour chaque feature
norm_df = pd.DataFrame(df_features['_score_norm'].tolist(), columns=feature_cols_for_score)
for i, col in enumerate(feature_cols_for_score):
    df_features[f'norm_{col}'] = norm_df[col]

# pondérations par défaut — ajustables
weights = {
    'mtp': 0.4,            # global pour mtp (moyenne des 3)
    'experience': 0.15,
    'job_match': 0.35,
    'vision': 0.10
}

# calcul final heuristique
df_features['mtp_mean_norm'] = df_features[['norm_mtp_metier_score','norm_mtp_talent_score','norm_mtp_paradigme_score']].mean(axis=1)
df_features['heuristic_score'] = (
    weights['mtp'] * df_features['mtp_mean_norm'] +
    weights['experience'] * df_features['norm_years_experience'] +
    weights['job_match'] * df_features['norm_sim_with_job'] +
    weights['vision'] * df_features['norm_sim_with_vision']
)

# 10) Chemin supervisé si label disponible
# chercher une colonne label évidente
label_col = None
for candidate in ['hired','hiring_result','status','result','is_hired']:
    if candidate in df_features.columns:
        label_col = candidate
        break

trained_model = None
if label_col is not None:
    # tenter d'entraîner un modèle supervisé
    print(f"Label supervisé trouvé: {label_col} — entraînement d'un classifieur")
    # préparer y
    y = df_features[label_col]
    # binariser si nécessaire
    if y.dtype == 'O' or y.dtype.name.startswith('category'):
        le = LabelEncoder()
        y_enc = le.fit_transform(y.fillna(''))
    else:
        y_enc = y.fillna(0).astype(int)
    X = df_features[[f'norm_{c}' for c in feature_cols_for_score]]
    X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, random_state=42, stratify=y_enc if len(set(y_enc))>1 else None)
    clf = RandomForestClassifier(n_estimators=200, random_state=42)
    try:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(classification_report(y_test, y_pred))
        trained_model = clf
        joblib.dump((clf, le), 'candidate_ranking_model.joblib')
        print('Modèle sauvegardé: candidate_ranking_model.joblib')
    except Exception as e:
        print('Erreur entraînement supervisé:', e)
else:
    print('Aucun label supervisé détecté — utilisation du scoring heuristique')

# 11) Fonction publique pour scorer et retourner top N

def score_and_rank(df=df_features, top_n=20, use_supervised=False):
    df = df.copy()
    if use_supervised and trained_model is not None:
        clf, le = trained_model, None
        try:
            # si on a sauvegardé le tuple
            loaded = joblib.load('candidate_ranking_model.joblib')
            if isinstance(loaded, tuple):
                clf, le = loaded
        except Exception:
            clf = trained_model
        X = df[[f'norm_{c}' for c in feature_cols_for_score]]
        preds = clf.predict_proba(X) if hasattr(clf, 'predict_proba') else clf.predict(X)
        # si probabilités
        if hasattr(clf, 'predict_proba'):
            df['supervised_score'] = preds[:, 1] if preds.shape[1] > 1 else preds[:, 0]
        else:
            df['supervised_score'] = preds
        df['final_score'] = df['supervised_score']
    else:
        df['final_score'] = df['heuristic_score']

    return df.sort_values('final_score', ascending=False).head(top_n)

# 12) Exemple d'utilisation
print('Nombre de candidatures:', len(df_features))
print('Top 5 candidats par scoring heuristique:')
print(score_and_rank(df_features, top_n=5)[['id','user_id','final_score']])

# exposer df_features pour inspection
candidate_scores = df_features[['id','user_id','candidate_text','heuristic_score','sim_with_job','sim_with_vision','years_experience','age']].copy()
candidate_scores = candidate_scores.rename(columns={'heuristic_score':'final_score'})

# sauvegarde optionnelle
candidate_scores.to_csv('candidate_scores.csv', index=False)
print('Fichier candidate_scores.csv généré')

In [ ]:
# Inspection et parsing de la colonne `mtp_answers` du DataFrame `df_app`
import json
import ast
from collections import defaultdict

print('Présence de la colonne mtp_answers dans df_app:', 'mtp_answers' in df_app.columns)
if 'mtp_answers' in df_app.columns:
    s = df_app['mtp_answers']
    print('Type pandas:', s.dtype)
    print('Nombre non-nuls:', s.notna().sum(), '/', len(s))
    display(s.head(10))

    # Échantillons uniques (limité)
    print('\nExemples (up to 10 unique values):')
    uniques = s.dropna().astype(str).unique()[:10]
    for i,u in enumerate(uniques,1):
        print(f'--- exemple {i} ---')
        print(u[:1000])
        print()

    # Fonction de parsing robuste
    def parse_mtp_answers(raw):
        """Retourne un dict plat {question: answer} en essayant plusieurs stratégies."""
        if raw is None or (isinstance(raw, float) and pd.isna(raw)):
            return {}
        # déjà dict
        if isinstance(raw, dict):
            return raw
        # chaine
        text = str(raw).strip()
        if text == '':
            return {}
        # essayer JSON
        try:
            parsed = json.loads(text)
            # parsed peut être liste ou dict
            if isinstance(parsed, dict):
                return parsed
            if isinstance(parsed, list):
                # si liste de paires ou liste de dicts
                out = {}
                for item in parsed:
                    if isinstance(item, dict):
                        out.update(item)
                    elif isinstance(item, (list, tuple)) and len(item) >= 2:
                        out[item[0]] = item[1]
                    else:
                        # si élément simple, index it
                        out.setdefault('answer_list', []).append(item)
                return out
        except Exception:
            pass
        # essayer ast.literal_eval
        try:
            parsed = ast.literal_eval(text)
            if isinstance(parsed, dict):
                return parsed
            if isinstance(parsed, list):
                out = {}
                for item in parsed:
                    if isinstance(item, dict):
                        out.update(item)
                return out
        except Exception:
            pass
        # dernier recours: chercher paires 'key: value' avec séparateurs
        out = {}
        # formats 'key=value' ou 'key: value' séparés par ; or | or , or \n
        for sep in [';','|','\n',',']:
            if sep in text and (':' in text or '=' in text):
                parts = [p.strip() for p in text.split(sep) if p.strip()]
                for p in parts:
                    if ':' in p:
                        k,v = p.split(':',1)
                        out[k.strip()] = v.strip()
                    elif '=' in p:
                        k,v = p.split('=',1)
                        out[k.strip()] = v.strip()
                if out:
                    return out
        # si encore rien, renvoyer texte brut sous clé raw_text
        return {'raw_text': text}

    # Appliquer et normaliser les clés MTP attendues
    parsed_series = s.apply(parse_mtp_answers)

    # Déterminer toutes les clés présentes
    all_keys = set()
    for d in parsed_series.dropna().tolist():
        if isinstance(d, dict):
            all_keys.update(d.keys())
    print('\nClés détectées dans mtp_answers (extrait):', list(sorted(all_keys))[:50])

    # Noms cibles qu'on veut mapper (exemple)
    target_keys = [
        'mtp_metier_q1','mtp_metier_q2','mtp_metier_q3',
        'mtp_talent_q1','mtp_talent_q2','mtp_talent_q3',
        'mtp_paradigme_q1','mtp_paradigme_q2','mtp_paradigme_q3'
    ]

    # Créer colonnes pour ces clés en essayant de trouver équivalents via correspondances simplifiées
    def find_value_for_key(d, key):
        # recherche directe
        if key in d:
            return d[key]
        # recherche par suffixe ou sans underscore
        simple = key.replace('_','').lower()
        for k in d.keys():
            if k is None:
                continue
            ks = str(k).replace('_','').lower()
            if simple == ks or simple in ks or ks in simple:
                return d[k]
        return None

    # Construire DataFrame de colonnes extraites
    extracted = defaultdict(list)
    for d in parsed_series:
        if not isinstance(d, dict):
            d = {}
        for k in target_keys:
            extracted[k].append(find_value_for_key(d, k))
        # aussi extraire texte combiné si disponible
        combined_text = []
        for val in d.values():
            if isinstance(val, (str,int,float)):
                combined_text.append(str(val))
        extracted['mtp_answers_text'].append('\n'.join(combined_text) if combined_text else None)

    df_mtp_parsed = pd.DataFrame(extracted, index=df_app.index)

    # Concatener au df_app
    for col in df_mtp_parsed.columns:
        df_app[col] = df_mtp_parsed[col]

    # --- Nouveau: conserver les réponses textuelles brutes par question ---
    # Créer colonnes *_text qui contiennent la réponse originale (chaîne) pour chaque question
    for k in target_keys:
        text_col = f"{k}_text"
        # garder la valeur originale sous forme de chaîne si présente
        df_app[text_col] = df_mtp_parsed[k].where(df_mtp_parsed[k].notna(), None).astype(object)

    print('\nColonnes ajoutées au DataFrame df_app :', list(df_mtp_parsed.columns) + [f"{k}_text" for k in target_keys])
    display_cols = ['id','mtp_answers'] + list(df_mtp_parsed.columns) + [f"{k}_text" for k in target_keys]
    # limiter l'affichage si trop large
    display(df_app[display_cols].head(10))

    # convertir les valeurs numériques si possible (en conservant les colonnes texte)
    for k in target_keys:
        try:
            # garder la colonne originale (texte) sous k + '_text', et stocker la version numérique sous la colonne k
            df_app[k] = pd.to_numeric(df_app[k], errors='coerce')
        except Exception:
            pass

    print('\nAperçu des statistiques numériques pour les questions MTP (colonnes numériques):')
    display(df_app[target_keys].describe())

else:
    print('La colonne mtp_answers n\'existe pas dans df_app')

Présence de la colonne mtp_answers dans df_app: True
Type pandas: object
Nombre non-nuls: 26 / 26


0    {'metier': ['- Ingénierie énergétique
- Mainte...
1    {'metier': ['Secteur : Énergie (B2B, B2C, syst...
2    {'metier': ['les problématiques se trouvent au...
3    {'metier': ['La méconnaissance des installatio...
4    {'metier': ['- sur le plan technique:
Les prob...
5    {'metier': ['1/Mauvaise gestion du capital hum...
6    {'metier': ['- Connaissances et maitrise des r...
7    {'metier': ['Insuffisance de la la production ...
8    {'metier': ['1.	Problématiques clés identifiée...
9    {'metier': ['les problématiques que j'identifi...
Name: mtp_answers, dtype: object


Exemples (up to 10 unique values):
--- exemple 1 ---
{'metier': ['- Ingénierie énergétique\n- Maintenance des bâtiments\n- Sécurité et sûreté\n- Logistique et achats\n- Gestion de projet, planification et budget, gestion des délais\n- Normes QHSE\n- Bonne maitrise du Pack OFFICE et de logiciels spécifiques \n- Gestion du patrimoine\n- Management d’équipe et conduite du changement \n- Sens du relationnel', 'Possédant une riche expérience professionnelle de vingt (20) ans dans les métiers des Moyens Généraux, Environnement de travail, Achats, QHSE, avec un fort relent satisfaction clientèle. J’ai appris à concilier rigueur, diplomatie, disponibilité, management et sens des relations commerciales tout en respectant les valeurs humaines.\n\nFort de cette expérience, j’ai acquis une solide expertise dans le développement de stratégies globales, la gestion financière, le management d’équipes et le marketing. J’ai pu mettre en place des processus opérationnels efficaces et optimiser les perf

,id,mtp_answers,mtp_metier_q1,mtp_metier_q2,mtp_metier_q3,mtp_talent_q1,mtp_talent_q2,mtp_talent_q3,mtp_paradigme_q1,mtp_paradigme_q2,mtp_paradigme_q3,mtp_answers_text
0,c60e6947-4b66-460b-b4ea-69229e193b90,{'metier': ['- Ingénierie énergétique - Mainte...,[- Ingénierie énergétique\n- Maintenance des b...,[- Ingénierie énergétique\n- Maintenance des b...,[- Ingénierie énergétique\n- Maintenance des b...,[- Forte capacité d'analyse et de prise de déc...,[- Forte capacité d'analyse et de prise de déc...,[- Forte capacité d'analyse et de prise de déc...,[- Compétences relationnelles\n- Création d'un...,[- Compétences relationnelles\n- Création d'un...,[- Compétences relationnelles\n- Création d'un...,None
1,be95722d-8219-4b0e-acc9-50d032952b79,"{'metier': ['Secteur : Énergie (B2B, B2C, syst...","[Secteur : Énergie (B2B, B2C, système énergéti...","[Secteur : Énergie (B2B, B2C, système énergéti...","[Secteur : Énergie (B2B, B2C, système énergéti...",[La capacité d'adaptation. Je pense que c'est ...,[La capacité d'adaptation. Je pense que c'est ...,[La capacité d'adaptation. Je pense que c'est ...,[Ce qui me motive le plus dans ma carrière c'e...,[Ce qui me motive le plus dans ma carrière c'e...,[Ce qui me motive le plus dans ma carrière c'e...,None
2,6f0a3290-0204-4662-8902-3991c8fcb0ab,{'metier': ['les problématiques se trouvent au...,[les problématiques se trouvent au niveau de ...,[les problématiques se trouvent au niveau de ...,[les problématiques se trouvent au niveau de ...,[le taux de recouvrement comprends deux élémen...,[le taux de recouvrement comprends deux élémen...,[le taux de recouvrement comprends deux élémen...,[S'assurer que la facturation de la SEEG a été...,[S'assurer que la facturation de la SEEG a été...,[S'assurer que la facturation de la SEEG a été...,None
3,ddebdf20-22ce-4da9-a490-c94516fb41fe,{'metier': ['La méconnaissance des installatio...,[La méconnaissance des installations de produc...,[La méconnaissance des installations de produc...,[La méconnaissance des installations de produc...,[Voici les étapes pour structurer un plan de m...,[Voici les étapes pour structurer un plan de m...,[Voici les étapes pour structurer un plan de m...,[L'innovation technique joue un rôle très impo...,[L'innovation technique joue un rôle très impo...,[L'innovation technique joue un rôle très impo...,None
4,d8321fde-fe23-4f10-863e-3102e625c14b,{'metier': ['- sur le plan technique: Les prob...,[- sur le plan technique:\nLes problématique r...,[- sur le plan technique:\nLes problématique r...,[- sur le plan technique:\nLes problématique r...,[La stratégie et les orientations générales so...,[La stratégie et les orientations générales so...,[La stratégie et les orientations générales so...,[La valeur de la proximité avec les usagers da...,[La valeur de la proximité avec les usagers da...,[La valeur de la proximité avec les usagers da...,None
5,e89bab46-05ff-4898-842d-2d002d8b2d80,{'metier': ['1/Mauvaise gestion du capital hum...,[1/Mauvaise gestion du capital humain\n2/Dette...,[1/Mauvaise gestion du capital humain\n2/Dette...,[1/Mauvaise gestion du capital humain\n2/Dette...,[1/ En optimisant le capital humain- (meilleur...,[1/ En optimisant le capital humain- (meilleur...,[1/ En optimisant le capital humain- (meilleur...,"[C'est un rôle centrale et important., il faut...","[C'est un rôle centrale et important., il faut...","[C'est un rôle centrale et important., il faut...",None
6,856a2c7d-4418-44c5-ab49-c29bfc8842d6,{'metier': ['- Connaissances et maitrise des r...,[- Connaissances et maitrise des risques profe...,[- Connaissances et maitrise des risques profe...,[- Connaissances et maitrise des risques profe...,[connaissance de la règlementation et de la no...,[connaissance de la règlementation et de la no...,[connaissance de la règlementation et de la no...,"[les meilleures conditions de travail., Lectur...","[les meilleures conditions de travail., Lectur...","[les meilleures conditions de travail., Lectur...",None
7,3fdf0


Aperçu des statistiques numériques pour les questions MTP:


,mtp_metier_q1,mtp_metier_q2,mtp_metier_q3,mtp_talent_q1,mtp_talent_q2,mtp_talent_q3,mtp_paradigme_q1,mtp_paradigme_q2,mtp_paradigme_q3
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_app.head()

,id,candidate_id,job_offer_id,status,reference_contacts,mtp_answers,mtp_metier_q1,mtp_metier_q2,mtp_metier_q3,mtp_talent_q1,mtp_talent_q2,mtp_talent_q3,mtp_paradigme_q1,mtp_paradigme_q2,mtp_paradigme_q3,mtp_answers_text
0,c60e6947-4b66-460b-b4ea-69229e193b90,4fa0fad9-d98f-4849-9ad4-68def0ed2074,f426f59a-97b5-4bd1-b260-165dec5362bf,candidature,Laurent BARRERE\nDirecteur Général de SOGAFRIC...,{'metier': ['- Ingénierie énergétique - Mainte...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,be95722d-8219-4b0e-acc9-50d032952b79,9f259121-b516-417f-8087-51a2fdf6d962,41112afb-d778-4c2b-bb86-f9ee3b6ca5a4,candidature,"Damien Madeleine, CAPGEMINI et Gérant de socié...","{'metier': ['Secteur : Énergie (B2B, B2C, syst...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,6f0a3290-0204-4662-8902-3991c8fcb0ab,4dc6bf49-7813-49fd-a764-b8a8fbe33995,c34e8321-c9ab-4884-902d-ce19d977bd90,candidature,,{'metier': ['les problématiques se trouvent au...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,ddebdf20-22ce-4da9-a490-c94516fb41fe,2f11d213-49f8-4407-ae5b-22cc99d3706b,737260aa-b155-443f-b21d-58bcbf8d151a,candidature,,{'metier': ['La méconnaissance des installatio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,d8321fde-fe23-4f10-863e-3102e625c14b,1e503472-4003-4941-ad30-8b1418f80170,707a8b92-4043-44c9-8a52-6c5b09cbb74c,candidature,,{'metier': ['- sur le plan technique: Les prob...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [ ]:
# Diagnostic avancé pour `df_app['mtp_answers']` et extraction par regex / nested JSON
import re
from itertools import islice

print('Colonne mtp_answers presente:', 'mtp_answers' in df_app.columns)
if 'mtp_answers' in df_app.columns:
    s = df_app['mtp_answers']
    nonnull = s.dropna()
    print('Non-null:', len(nonnull), 'total:', len(s))
    # Afficher les 5 premières valeurs brutes longues
    print('\n--- 5 premiers exemples bruts (tronqués à 200 chars) ---')
    for i, val in enumerate(islice(nonnull.astype(str).tolist(), 5)):
        print(f'[{i+1}] ({len(val)} chars) ->', val[:200].replace('\n','\\n'))
        print('-----')

    # Fonction d'extraction plus agressive
    def extract_mtp_values(raw):
        if raw is None or (isinstance(raw, float) and pd.isna(raw)):
            return {}
        text = str(raw)
        out = {}
        # 1) tenter JSON/literal_eval (réutiliser parse logic simple)
        try:
            parsed = json.loads(text)
        except Exception:
            try:
                parsed = ast.literal_eval(text)
            except Exception:
                parsed = None
        if parsed is not None:
            # si dict plat
            if isinstance(parsed, dict):
                # normaliser keys to strings
                for k,v in parsed.items():
                    out[str(k).strip()] = v
                # essayer trouver nested lists of answers
                # ex: {'answers':[{'question':'mtp_metier_q1','answer':3}, ...]}
                if 'answers' in parsed and isinstance(parsed['answers'], (list,tuple)):
                    for a in parsed['answers']:
                        if isinstance(a, dict):
                            q = a.get('question') or a.get('key') or a.get('name')
                            v = a.get('answer') or a.get('value') or a.get('response')
                            if q:
                                out[str(q)] = v
            elif isinstance(parsed, list):
                # liste d'items potentiellement dicts
                for item in parsed:
                    if isinstance(item, dict):
                        for k,v in item.items():
                            out[str(k).strip()] = v
        # 2) extraction par regex pour mtp_* patterns
        # rechercher patterns comme metier q1: 4 ou mtp_metier_q1=4 ou "mtp_metier_q1": "4"
        patterns = [r"(mtp[_\s-]*metier[_\s-]*q?\s*1)[\s:\=\"]*([0-9]+)",
                    r"(mtp[_\s-]*metier[_\s-]*q?\s*2)[\s:\=\"]*([0-9]+)",
                    r"(mtp[_\s-]*metier[_\s-]*q?\s*3)[\s:\=\"]*([0-9]+)",
                    r"(metier[_\s-]*q?\s*1)[\s:\=\"]*([0-9]+)",
                    r"(mtp[_\s-]*talent[_\s-]*q?\s*1)[\s:\=\"]*([0-9]+)",
                    r"(talent[_\s-]*q?\s*1)[\s:\=\"]*([0-9]+)",
                    r"(paradigme[_\s-]*q?\s*1)[\s:\=\"]*([0-9]+)"]
        for pat in patterns:
            for m in re.finditer(pat, text, flags=re.IGNORECASE):
                k = re.sub(r"[^a-zA-Z0-9_]","_", m.group(1).lower())
                out[k] = m.group(2)
        # 3) rechercher général de type 'q1: 4' proche des mots metier/talent/paradigme
        general = re.finditer(r"(metier|talent|paradigme)[^\n\r]{0,30}?(?:q?\s*([1-3]))[^0-9]*([0-9])", text, flags=re.IGNORECASE)
        for m in general:
            base = m.group(1).lower()
            qnum = m.group(2)
            val = m.group(3)
            key = f"mtp_{base}_q{qnum}"
            out[key] = val
        # 4) trouver nombres isolés 1-5 if context contains 'mtp' word
        if 'mtp' in text.lower() and not any(k.startswith('mtp') for k in out.keys()):
            nums = re.findall(r"\b([1-5])\b", text)
            # heuristique : assigner en ordre si 3 valeurs
            if len(nums) >= 3:
                out['mtp_metier_q1'] = nums[0]
                out['mtp_metier_q2'] = nums[1]
                out['mtp_metier_q3'] = nums[2]
        # 5) fallback: store raw
        if not out:
            out['raw_text'] = text
        return out

    # appliquer extraction agressive et montrer premiers résultats
    diag = nonnull.apply(extract_mtp_values)
    print('\nDiagnostics pour les 8 premiers éléments non-nuls:')
    for i, d in enumerate(diag.tolist()[:8],1):
        print(f'--- item {i} ---')
        print(d)
    
    # construire colonnes cibles à partir du diag
    target_keys = ['mtp_metier_q1','mtp_metier_q2','mtp_metier_q3','mtp_talent_q1','mtp_talent_q2','mtp_talent_q3','mtp_paradigme_q1','mtp_paradigme_q2','mtp_paradigme_q3']
    extracted = {k: [] for k in target_keys}
    extracted['raw_parsed'] = []
    # remplacer iteritems() (obsolète) par items()
    for idx, raw in s.items():
        parsed = extract_mtp_values(raw)
        for k in target_keys:
            extracted[k].append(parsed.get(k) or parsed.get(k.replace('mtp_','')) or parsed.get(k.replace('_','')))
        extracted['raw_parsed'].append(parsed)
    df_mtp = pd.DataFrame(extracted, index=df_app.index)
    # attacher
    for col in df_mtp.columns:
        df_app[col] = df_mtp[col]
    print('\nColonnes tentées ajoutées:', list(df_mtp.columns))
    print('\nStatuts d\'existence:')
    for k in target_keys:
        print(k, 'non-null count ->', df_app[k].notna().sum())
    
    display(df_app[['id'] + target_keys].head(10))
else:
    print('La colonne mtp_answers n\'est pas présente.')

Colonne mtp_answers presente: True
Non-null: 26 total: 26

--- 5 premiers exemples bruts (tronqués à 200 chars) ---
[1] (4573 chars) -> {'metier': ['- Ingénierie énergétique\n- Maintenance des bâtiments\n- Sécurité et sûreté\n- Logistique et achats\n- Gestion de projet, planification et budget, gestion des délais\n- Normes QHSE\n- Bon
-----
[2] (10131 chars) -> {'metier': ['Secteur : Énergie (B2B, B2C, système énergétique), Télécom (B2C, Billing, BI/Big Data), E-commerce (B2C), Fonction Publique\nDomaines : ERP (Commercialisation, Distribution et Facturation
-----
[3] (3814 chars) -> {'metier': ["les problématiques se trouvent au niveau  de la facturation, du contrôle installation du recouvrement . au niveau de la facturation; sassurer que tout compteur posé, ramène une consommati
-----
[4] (13254 chars) -> {'metier': ["La méconnaissance des installations de production et distribution d'eau qui se traduit par la non prise en charge des véritables problèmes. La qualification du manager 

,id,mtp_metier_q1,mtp_metier_q2,mtp_metier_q3,mtp_talent_q1,mtp_talent_q2,mtp_talent_q3,mtp_paradigme_q1,mtp_paradigme_q2,mtp_paradigme_q3
0,c60e6947-4b66-460b-b4ea-69229e193b90,None,None,None,None,None,None,None,None,None
1,be95722d-8219-4b0e-acc9-50d032952b79,None,2,None,None,None,None,None,None,None
2,6f0a3290-0204-4662-8902-3991c8fcb0ab,None,None,None,None,None,None,None,None,None
3,ddebdf20-22ce-4da9-a490-c94516fb41fe,None,None,None,None,None,None,None,None,None
4,d8321fde-fe23-4f10-863e-3102e625c14b,None,None,None,None,None,None,None,None,None
5,e89bab46-05ff-4898-842d-2d002d8b2d80,2,None,None,2,None,None,None,None,None
6,856a2c7d-4418-44c5-ab49-c29bfc8842d6,None,None,None,None,None,None,None,None,None
7,3fdf0cf6-5efb-426f-80ba-b86c9e0a7833,None,None,None,None,None,None,None,None,None
8,dd1dd88c-e970-46b0-93c7-9cae866458c6,6,None,None,None,None,None,None,None,None
9,834cca94-a92e-4cf0-9f14-bbf16814313b,None,None,None,None,None,None,None,None,None
